# Fit, Position, and Aspect Ratios in Editor
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/feature/fit_position_aspect_ratios.ipynb)

---

This notebook is a runnable, visual walkthrough of how **Editor** places a video onto a timeline when the **timeline aspect ratio** doesn’t match the **video aspect ratio**.

You’ll learn (by rendering real outputs):
- How the same video behaves on **16:9**, **9:16**, and **1:1** timelines
- How each fit mode changes scaling and framing:
  - `Fit.crop`
  - `Fit.contain`
  - `Fit.cover`
  - `Fit.none` (or `fit=None`)
- How `position` and `offset` let you **reframe** a clip *without stretching* the video

We’ll keep the video the same, and only change the **timeline resolution** and **Clip fit/position/offset** so the behavior is easy to compare.

---
## 📦 Step 1: Install dependencies

Lets install the VideoDB SDK

In [ ]:
%pip -q install videodb

  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Run the next cell and enter your `VIDEO_DB_API_KEY` when prompted.

In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---
## 📦 Step 3: Connect to a collection

A **collection** is where your uploaded assets live (videos, images, audio).
We’ll upload one video next and reuse it throughout this notebook.

In [ ]:
coll = conn.get_collection()

---
## 📦 Step 4: Upload a video asset

Editor works with **assets stored in VideoDB**.
Here we upload a YouTube URL as a VideoDB video asset. The returned object has an `id` we’ll pass into `VideoAsset(id=...)`.

> Tip: When you re-run a notebook, you can skip re-uploading and fetch an existing asset by ID instead (see the commented snippet below).

In [ ]:
# Upload any asset by url
video = coll.upload(url="https://www.youtube.com/watch?v=RB9nyUyNI2s")
print("Uploaded video asset:", video.id)

# Optional re-run pattern (don’t upload again):
# video = coll.get_video("video_id")

Uploaded video asset: m-z-019b49b9-8618-7510-9cc7-6e0133c88bf1


In [ ]:
# Uploading vertical video for 16:9 resolution Fit testing
video_vertical = coll.upload("https://www.youtube.com/shorts/XCmVyeUKwus")
print("Uploaded video asset:", video_vertical.id)

# Optional re-run pattern (don’t upload again):
# video_vertical = coll.get_video("video_id")

Uploaded video asset: m-z-019b4d70-f905-7a83-af98-1d82ac0a52e5


---
## 📦 Step 5: Import Editor building blocks

We’ll use a small set of Editor objects:
- `Timeline`: the global canvas (resolution + background)
- `Track`: a layer on the timeline
- `Clip`: a container that controls how an asset appears on the timeline
- `VideoAsset`: references your uploaded video by `id`
- `Fit`, `Position`, `Offset`: the tools we’re exploring in this notebook

In [ ]:
from videodb import play_stream
from videodb.editor import Timeline, Track, Clip, VideoAsset, Fit, Position, Offset

---
## 📦 Step 6: Define the three timeline canvases (16:9, 9:16, 1:1)

A timeline’s `resolution` is the **viewport** your video must fit inside.
We’ll render the *same* video onto three different canvas shapes.

We’ll also use a neutral gray background so letterboxing/cropping is easy to see.

In [ ]:
RES_16_9 = "1280x720"
RES_9_16 = "608x1080"
RES_1_1 = "1080x1080"
BACKGROUND = "#808080"

fit_none = Fit.none

print("16:9 ->", RES_16_9)
print("9:16 ->", RES_9_16)
print("1:1 ->", RES_1_1)
print("background ->", BACKGROUND)
print("Using fit for no-scaling ->", fit_none)

16:9 -> 1280x720
9:16 -> 608x1080
1:1 -> 1080x1080
background -> #808080
Using fit for no-scaling -> None


---
## 📦 Step 7: The mental model (why Fit exists)

Think of the timeline as a **window** (the output frame) and your video as a **rectangle** with its own aspect ratio.

When the window and rectangle don’t match shapes, Editor must decide:
- Do we **crop**? (fill the window, lose edges)
- Do we **letterbox**? (show everything, accept bars)
- Do we **stretch**? (fill the window, distort)
- Do we do **no automatic scaling**? (native pixels)

That’s exactly what the fit modes represent.

---
## 📦 Step 8: Fit experiments on a 16:9 timeline (landscape)

Run the next cells one by one. Each cell renders exactly one output stream so it’s easy to compare.

### 16:9 + `Fit.crop`

Fills the 16:9 canvas by cropping if needed (no stretching).

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.crop)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_16_9_crop = timeline.generate_stream()
print(stream_16_9_crop)
play_stream(stream_16_9_crop)

https://play.videodb.io/v1/2afa9530-7cc2-4ffe-8da5-2970411b2770.m3u8


### 16:9 + `Fit.contain`

Shows the full video without cropping. If the source aspect ratio doesn’t match, you’ll see bars.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.contain)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_16_9_contain = timeline.generate_stream()
print(stream_16_9_contain)
play_stream(stream_16_9_contain)

https://play.videodb.io/v1/c7859b85-a155-410a-887f-d40e708eba88.m3u8


### 16:9 + `Fit.cover` (stretch)

Fills the entire canvas by stretching the video (aspect ratio is not preserved). Use sparingly.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.cover)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_16_9_cover = timeline.generate_stream()
print(stream_16_9_cover)
play_stream(stream_16_9_cover)

https://play.videodb.io/v1/1d702af5-bc02-4a8e-8b06-cbdedf182fd2.m3u8


### 16:9 + no scaling (`Fit.none` / `fit=None`)

Disables automatic scaling. The clip renders at its native pixel size, which may leave background visible or extend outside the canvas depending on the source.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.none)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_16_9_none = timeline.generate_stream()
print(stream_16_9_none)
play_stream(stream_16_9_none)

https://play.videodb.io/v1/b2b82fd7-bb8f-471b-b8c9-b6e74801a63a.m3u8


---
## 📦 Step 9: Fit experiments on a 9:16 timeline (portrait)

Run the next cells one by one. Each cell renders exactly one output stream so it’s easy to compare.

### 9:16 + `Fit.crop`

Fills the portrait canvas by cropping (no stretching). This often feels like a “zoom-in” on a landscape source.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_9_16

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=Fit.crop)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_9_16_crop = timeline.generate_stream()
print(stream_9_16_crop)
play_stream(stream_9_16_crop)

https://play.videodb.io/v1/e7a3596f-9f98-4272-a824-ad1342467d06.m3u8


### 9:16 + `Fit.contain`

Shows the full video without cropping. On a portrait canvas, a landscape source usually produces big top/bottom bars.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_9_16

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=Fit.contain)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_9_16_contain = timeline.generate_stream()
print(stream_9_16_contain)
play_stream(stream_9_16_contain)

https://play.videodb.io/v1/5535e871-d1bc-4289-bea6-e9d1c1815581.m3u8


### 9:16 + `Fit.cover` (stretch)

Fills the portrait canvas by stretching the video (distortion). This avoids bars and avoids cropping, but changes shapes.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_9_16

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=Fit.cover)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_9_16_cover = timeline.generate_stream()
print(stream_9_16_cover)
play_stream(stream_9_16_cover)

https://play.videodb.io/v1/9e478cd1-c729-4387-9591-54f0eeee7047.m3u8


### 9:16 + no scaling (`Fit.none` / `fit=None`)

Disables automatic scaling. Useful for precise layouts, but you may see a lot of background (or clipped edges) depending on the source.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_9_16

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=fit_none)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_9_16_none = timeline.generate_stream()
print(stream_9_16_none)
play_stream(stream_9_16_none)

https://play.videodb.io/v1/62f6ef29-fde4-4008-98f8-bbe632a232ed.m3u8


---
## 📦 Step 10: Fit experiments on a 1:1 timeline (square)

Run the next cells one by one. Each cell renders exactly one output stream so it’s easy to compare.

### 1:1 + `Fit.crop`

Fills the square canvas by cropping if needed (no stretching).

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_1_1

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=Fit.crop)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_1_1_crop = timeline.generate_stream()
print(stream_1_1_crop)
play_stream(stream_1_1_crop)

https://play.videodb.io/v1/98b1acc7-7f20-49f1-99d8-f5fc7219559a.m3u8


### 1:1 + `Fit.contain`

Shows the full video without cropping. If aspect ratios don’t match, you’ll see bars.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_1_1

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=Fit.contain)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_1_1_contain = timeline.generate_stream()
print(stream_1_1_contain)
play_stream(stream_1_1_contain)

https://play.videodb.io/v1/a4562c2c-63a0-485e-bb09-690eb115d325.m3u8


### 1:1 + `Fit.cover` (stretch)

Fills the square canvas by stretching the video (distortion). Use sparingly.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_1_1

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=Fit.cover)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_1_1_cover = timeline.generate_stream()
print(stream_1_1_cover)
play_stream(stream_1_1_cover)

https://play.videodb.io/v1/45069371-0e02-4774-ad83-6813fd040121.m3u8


### 1:1 + no scaling (`Fit.none` / `fit=None`)

Disables automatic scaling (native pixels). You may see background or clipped edges depending on the source.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_1_1

clip = Clip(asset=VideoAsset(id=video.id), duration=6, fit=fit_none)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_1_1_none = timeline.generate_stream()
print(stream_1_1_none)
play_stream(stream_1_1_none)

https://play.videodb.io/v1/22dfb8f6-4acb-4e0d-a3b4-d2ffbf823711.m3u8


---
## 📦 Step 11: Position and Offset (reframing without stretching)

Run the next cells one by one. Each cell renders exactly one output stream so you can see how `position` and `offset` change framing without changing aspect ratio.

### Baseline: 16:9 + `Fit.contain` + `Position.center`

Start with the default portrait crop framing (centered).

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.contain, position=Position.center)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_pos_center = timeline.generate_stream()
print(stream_pos_center)
play_stream(stream_pos_center)

https://play.videodb.io/v1/f72a0b5f-ee07-41de-80db-1982f7b07fa9.m3u8


### Reframe: `Position.left`

Keeps more of the left side visible in a crop-based fit (you’re choosing what gets cropped, not stretching).

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.contain, position=Position.left)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_pos_left = timeline.generate_stream()
print(stream_pos_left)
play_stream(stream_pos_left)

https://play.videodb.io/v1/1b627e7a-c60c-4c58-822a-ac088f1aaa49.m3u8


### Reframe: `Position.right`

Keeps more of the right side visible in a crop-based fit.

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.contain, position=Position.right)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_pos_right = timeline.generate_stream()
print(stream_pos_right)
play_stream(stream_pos_right)

https://play.videodb.io/v1/33cac65d-c3d7-4985-b3b7-3948f95c0792.m3u8


### Fine-tune: `Offset(x=-0.25)`

Offsets are small nudges after scaling/positioning. This shifts the video left within the crop (no stretching).

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.contain, position=Position.center, offset=Offset(x=-0.25))

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_offset_x_left = timeline.generate_stream()
print(stream_offset_x_left)
play_stream(stream_offset_x_left)

https://play.videodb.io/v1/13f88e6d-7bfb-481f-9403-2676d8e17280.m3u8


### Fine-tune: `Offset(x=+0.25)`

Shifts the video right within the crop (no stretching).

In [ ]:
timeline = Timeline(conn)
timeline.background = BACKGROUND
timeline.resolution = RES_16_9

clip = Clip(asset=VideoAsset(id=video_vertical.id), duration=6, fit=Fit.contain, position=Position.center, offset=Offset(x=0.25))

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_offset_x_left = timeline.generate_stream()
print(stream_offset_x_left)
play_stream(stream_offset_x_left)

https://play.videodb.io/v1/76282711-d01b-4c5e-93e6-6a40e4d684f9.m3u8


---
## ✅ Wrap-up

You now have a practical mental model for aspect-ratio mismatches:

- **`Fit.crop`**: full-bleed frame, crops edges
- **`Fit.contain`**: shows everything, adds bars
- **`Fit.cover`**: fills by stretching (distortion — use sparingly)
- **`Fit.none` / `fit=None`**: no automatic scaling

And for framing without distortion:

- **`position`** chooses which part of the scaled video is prioritized
- **`offset`** fine-tunes that framing by sliding the video inside the canvas